In [2]:
print(1)

1


In [1]:
smiles = []
with open("raw/chemicals.v5.0.tsv", 'r+', encoding="ISO-8859-1") as f:
    for line in f:
        smiles.append(line.strip().split('\t')[3])

In [1]:
smiles[:5]

NameError: name 'smiles' is not defined

In [3]:
smiles = smiles[1:]

In [4]:
len(smiles)

116224359

In [5]:
def save_as_tsv(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for row in data:
            f.write('\t'.join(map(str, row)) + '\n')

def split_and_save_list(big_list, chunk_size=10000000):
    for i in range(0, len(big_list), chunk_size):
        chunk = big_list[i:i + chunk_size]
        filename = f'data2/chunk_{i//chunk_size}.tsv'
        save_as_tsv(chunk, filename)
        print(f'Saved {filename}')

split_and_save_list(smiles, len(smiles)//5)

Saved data2/chunk_0.tsv


KeyboardInterrupt: 

In [6]:
import gc
smiles = None
gc.collect()

513

## 분할된 tsv 불러와서 id로 매핑

In [1]:
comp2id = {}
with open("data/vocab_compound") as f:
    for index, line in enumerate(f):
        comp2id[line.strip()] = index
print(comp2id)

{'_PAD': 0, '_GO': 1, '_EOS': 2, '_UNK': 3, 'C': 4, '=': 5, '(': 6, ')': 7, 'O': 8, 'N': 9, '1': 10, '2': 11, '3': 12, '4': 13, '[': 14, ']': 15, 'S': 16, 'l': 17, 'F': 18, '-': 19, '5': 20, '+': 21, '.': 22, '6': 23, 'B': 24, 'r': 25, '#': 26, 'P': 27, 'i': 28, 'H': 29, '7': 30, 'I': 31, '8': 32, '9': 33, 'a': 34, 'e': 35, 'A': 36, 'n': 37, 's': 38, 'u': 39, 'g': 40, 'o': 41, 't': 42, 'T': 43, 'M': 44, 'Z': 45, 'b': 46, 'K': 47, 'R': 48, 'd': 49, 'W': 50, 'G': 51, 'L': 52, 'c': 53, 'h': 54, 'V': 55, 'm': 56, 'E': 57, 'Y': 58, 'U': 59, 'f': 60, 'D': 61, 'y': 62, '%': 63, '0': 64, 'p': 65, 'k': 66, 'X': 67}


In [2]:
import torch

def insert_padding(mylist, max_size):
    for i, sentence in enumerate(mylist):
        templist = [0 for _ in range(max_size-len(sentence))]
        mylist[i] = mylist[i] + templist
    return mylist

def process_independent(mylist, max_size, file_name):
    mylist = insert_padding(mylist, max_size)
    mytensor = torch.tensor(mylist)
    torch.save(mytensor, 'data2/'+file_name+".pt")
    return mytensor

comp_MAX_size = 100

In [3]:
from tqdm import tqdm
import numpy as np

for file_num in range(6):  # 0부터 5까지 파일 번호
    file_path = f'data2/chunk_{file_num}.tsv'
    smiles_ids = []  # 현재 파일의 결과를 저장할 리스트

    with open(file_path, 'r') as f:
        for line in tqdm(f, desc=f"Processing chunk {file_num}"):
            sentence = line.strip().split('\t')
            temp = []
            skip_string = False
            for char in sentence:
                if char not in comp2id:
                    skip_string = True
                    break
                else:
                    temp.append(comp2id[char])
            if not skip_string:
                smiles_ids.append(temp)

    process_independent(smiles_ids, comp_MAX_size, f"data2/smiles_ids_{file_num}")

    # 파일 처리 후 메모리 관리
    smiles_ids = []
    gc.collect()  # 파일 하나 처리 후 강제 가비지 컬렉션


Processing chunk 0: 23244871it [10:16, 37678.99it/s]


RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 18595896800 bytes. Error code 12 (Cannot allocate memory)